In [ ]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load Cora citation network dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class BetterGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # First GCN layer → increases feature dimension to 16
        self.conv1 = GCNConv(data.num_node_features, 16)

        # Second hidden layer → deeper graph feature extraction
        self.conv2 = GCNConv(16, 16)

        # Output layer → maps to number of classes for classification
        self.conv3 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Layer 1 → graph aggregation + ReLU activation
        x = F.relu(self.conv1(x, edge_index))

        # Dropout helps prevent overfitting
        x = F.dropout(x, p=0.5, training=self.training)

        # Layer 2 → deeper learning of graph structure
        x = F.relu(self.conv2(x, edge_index))

        # Another dropout layer to improve generalization
        x = F.dropout(x, p=0.5, training=self.training)

        # Final GCN layer → class logits for each node
        x = self.conv3(x, edge_index)

        # Log-softmax for node classification
        return F.log_softmax(x, dim=1)

model = BetterGCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

model.eval()
pred = model(data).argmax(dim=1)

acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()

print(f"Improved GCN Test Accuracy: {acc:.2f}")


Improved GCN Test Accuracy: 0.80


In [ ]:
!pip install torch_geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.4 MB/s eta 0:00:00
